# Web Scraping: CNMV

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os

---
## Scraping de la CNMV

### ¿Qué es la CNMV?

La **Comisión Nacional del Mercado de Valores** (cnmv.es) supervisa los mercados financieros en España. Su web contiene registros públicos con información sobre:

- Sociedades y Agencias de Valores registradas
- Empresas de Asesoramiento Financiero
- Entidades advertidas ("chiringuitos financieros")
- Hechos relevantes de empresas cotizadas

### Objetivo

Vamos a extraer el **listado de Sociedades y Agencias de Valores** registradas en la CNMV, obteniendo:
- Nombre de la entidad
- Número de registro
- Fecha de registro
- Dirección

### 1.1 Descargar la página del listado

In [ ]:
# URL del listado de Sociedades y Agencias de Valores
url_cnmv = "https://www.cnmv.es/portal/consultas/listadoentidad?id=1&tipoent=0&lang=es"

# La CNMV necesita headers para responder correctamente
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept-Language": "es-ES,es;q=0.9",
}

respuesta = requests.get(url_cnmv, headers=headers)
print(f"Status code: {respuesta.status_code}")
print(f"Tamaño: {len(respuesta.text):,} caracteres") # Las {} son marcadores de expresión dentro del f-string

In [ ]:
soup = BeautifulSoup(respuesta.text, "html.parser")

# Confirmamos que llegamos a la página correcta
titulo = soup.find("h1")
if titulo:
    print("Página:", titulo.text.strip())
else:
    print("Título de la página:", soup.title.text.strip() if soup.title else "no encontrado")

### 1.2 Analizar la estructura HTML

Antes de extraer datos, necesitamos **inspeccionar** el HTML para entender cómo están organizados.

> En tu navegador, haz clic derecho sobre un nombre de entidad → "Inspeccionar" para ver los tags HTML.

La página de la CNMV tiene bloques repetidos con esta estructura:
```
NOMBRE DE LA ENTIDAD
Número y fecha de registro oficial: 251 - 29/08/2013
Dirección: CALLE TAL, Nº X - CÓDIGO CIUDAD
```

Vamos a buscar los elementos que contienen esta información.

In [ ]:
# Buscamos el contenedor principal del contenido
# La CNMV usa ASP.NET, el contenido suele estar en un div con id "maincontent" o similar
main = soup.find("div", id="maincontent") or soup.find("main") or soup

# Exploramos: buscamos todos los textos que contengan "Número y fecha de registro"
# Esto nos ayuda a localizar los bloques de entidades
textos_registro = main.find_all(string=re.compile(r"Número y fecha de registro"))
print(f"Bloques con 'Número y fecha de registro' encontrados: {len(textos_registro)}")

# Contexto HTML para entender la estructura
if textos_registro:
    # Subimos al elemento padre para ver el bloque completo
    bloque = textos_registro[0].find_parent("div") or textos_registro[0].parent
    print("\nEstructura del primer bloque:")
    print(bloque.prettify()[:300])

### 1.3 Extraer los datos de cada entidad

Ahora que conocemos la estructura, vamos a extraer los datos de forma sistemática.

> **Nota:** Las webs institucionales pueden cambiar su estructura HTML sin aviso. Si los selectores no funcionan, habrá que inspeccionarla de nuevo. Esto es parte de la realidad del scraping.

In [ ]:
# Extraemos el texto completo del contenido principal
texto_completo = main.get_text(separator="\n")

# Usamos expresiones regulares para capturar los patrones de datos
# Patrón: líneas con "Número y fecha de registro oficial: NUM - DD/MM/AAAA"
patron_registro = re.compile(
    r"Número y fecha de registro oficial:\s*(\d+)\s*-\s*(\d{2}/\d{2}/\d{4})"
)

# Patrón: líneas con "Dirección: ..."
patron_direccion = re.compile(
    r"Dirección:\s*(.+)"
)

registros = patron_registro.findall(texto_completo)
direcciones = patron_direccion.findall(texto_completo)

print(f"Registros encontrados: {len(registros)}")
print(f"Direcciones encontradas: {len(direcciones)}")

# Mostramos los primeros 3
for num, fecha in registros[:3]:
    print(f"  Nº {num} - Fecha: {fecha}")

In [ ]:
# Ahora extraemos los nombres de las entidades
# Los nombres aparecen justo ANTES de cada "Número y fecha de registro"
# Dividimos el texto por ese patrón y cogemos la línea anterior

lineas = texto_completo.split("\n")
lineas = [l.strip() for l in lineas if l.strip()]  # Limpiamos vacías

entidades = []
for i, linea in enumerate(lineas):
    match = patron_registro.search(linea)
    if match:
        # El nombre de la entidad suele estar en la línea anterior
        nombre = lineas[i - 1] if i > 0 else "Desconocido"
        num_registro = match.group(1)
        fecha_registro = match.group(2)
        
        # La dirección suele estar en la línea siguiente
        direccion = ""
        if i + 1 < len(lineas):
            match_dir = patron_direccion.search(lineas[i + 1])
            if match_dir:
                direccion = match_dir.group(1).strip()
        
        entidades.append({
            "nombre": nombre,
            "num_registro": int(num_registro),
            "fecha_registro": fecha_registro,
            "direccion": direccion,
        })

print(f"{len(entidades)} entidades extraídas")
print("\nPrimeras 3:")
for e in entidades[:3]:
    print(f"  {e['nombre']} (Reg. {e['num_registro']}, {e['fecha_registro']})")

In [ ]:
df_cnmv = pd.DataFrame(entidades)
df_cnmv["fecha_registro"] = pd.to_datetime(df_cnmv["fecha_registro"], format="%d/%m/%Y")
df_cnmv = df_cnmv.sort_values("fecha_registro", ascending=False).reset_index(drop=True)
df_cnmv

### 1.4 Análisis rápido

In [ ]:
# ¿Cuántas entidades se registraron por año?
df_cnmv["anyo"] = df_cnmv["fecha_registro"].dt.year
print("Entidades registradas por año (últimos 10):")
print(df_cnmv["anyo"].value_counts().sort_index(ascending=True).tail(10).to_string())

# ¿En qué ciudades están?
# La dirección suele terminar en "CÓDIGO CIUDAD", extraemos la ciudad
df_cnmv["ciudad"] = df_cnmv["direccion"].str.extract(r"\d{5}\s+(.+)$")[0]
print("\nTop 5 ciudades:")
print(df_cnmv["ciudad"].value_counts().head().to_string())

In [ ]:
# URLs directas a los PDFs públicos de Iberdrola
pdfs = {
    "iberdrola_gei": "https://www.iberdrola.com/documents/20125/41101/informe-gei-2023.pdf",
    "iberdrola_indicadores": "https://www.iberdrola.com/documents/20125/3643974/informe-integrado-esg-2023-indicadores-clave-sostenibilidad.pdf",
}

# Creamos una carpeta para guardar los PDFs
os.makedirs("pdfs_sostenibilidad", exist_ok=True)

# Descargamos cada PDF
for nombre, url in pdfs.items():
    ruta = f"pdfs_sostenibilidad/{nombre}.pdf"
    
    if os.path.exists(ruta):
        print(f"  ✓ {nombre}.pdf ya existe, saltando descarga")
        continue
    
    print(f"  Descargando {nombre}...", end=" ")
    resp = requests.get(url, headers=headers, timeout=60)
    
    if resp.status_code == 200 and resp.headers.get("Content-Type", "").startswith("application/pdf"):
        with open(ruta, "wb") as f:
            f.write(resp.content)
        print(f"OK ({len(resp.content)/1024:.0f} KB)")
    else:
        print(f"ERROR (status {resp.status_code})")

# Verificamos qué tenemos
for f in os.listdir("pdfs_sostenibilidad"):
    size = os.path.getsize(f"pdfs_sostenibilidad/{f}") / 1024
    print(f"  📄 {f} ({size:.0f} KB)")